# Tabla de Contenido 

* [Introducción](#intro) 
* [Etapa 1. Exploración de los datos](#data_review)
* [Etapa 2. Transformación de datos](#data_preprocessing)
*[Etapa 3. Clasificación de datos](#data_classification)
* [Etapa 4. Comprobación de hipótesis](#hypotheses)
* [Etapa 5. Conclusión general](#end)

# Introducción <a id='intro'></a>

## Análisis del riesgo de incumplimiento de los prestatarios

El proyecto consiste en preparar un informe para la división de préstamos de un banco. Se averiguará si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

El informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

## Objetivos

1. Preprocesar los datos de la base de clientes del banco, para asegurar su calidad previo a el análisis de los mismos.

2. Análisis de los datos de clientes del banco para responder a pregutnas de investigación que permitan crear una puntuación de crédito para un cliente potencial. 

## Etapas

El proyecto consistirá en cinco etapas:

1. Descripción de los datos
2. Preprocesamiento de datos
3. Clasificación de datos
4. Prueba de hipótesis
5. Conclusión general

In [1]:
# Cargar todas las librerías

import pandas as pd

In [2]:
# Carga los datos

try:
    credit_scoring = pd.read_csv('credit_scoring_eng.csv')
except:
    credit_scoring = pd.read_csv('/datasets/credit_scoring_eng.csv')

# Etapa 1. Exploración de datos <a id='data_review'></a>

## Descripción de los datos 

- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

In [3]:
# Muestra cuántas filas y columnas tiene el conjunto de datos

credit_scoring.shape

(21525, 12)

In [4]:
# Muestra las primeras filas N

credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


Existen algunos problemas en los datos que se pueden identificar inicialmente:

1.- Los datos de experiencia laboral están días (y no en años que es lo usual), tienen valores negativos, y de coma flotante.


2.- La columna de educación contiene datos en letras mayúsculas y minúsculas, lo cual podría derivar en problemas si se trata de categorizar.


In [5]:
# Obtener información sobre los datos
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Existen valores ausentes en las columnas "days_employed" y "total_income".Los tipos de datos parecen estar acordes a la información que contiene cada columna.

In [6]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos

credit_scoring[credit_scoring["days_employed"].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Al parecer los valores ausentes de las columnas "days_employed" y "total_income" coinciden. 

In [7]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.

credit_scoring[credit_scoring["days_employed"].isna() & credit_scoring["total_income"].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [8]:
#  Calculo del porcentaje de los valores ausentes en comparación con el conjunto de datos completo

# valores_ausentes = credit_scoring.isna().sum()
# valores_totales = len(credit_scoring)
# porcentaje_ausentes = (valores_ausentes/valores_totales)*100
# porcentaje_ausentes

**Conclusión intermedia**

Los datos faltantes en experiencia "days_employed" e ingresos "total_income" se podría rellenar categorizando los valores de la columna tipo de empleo "income_type", y escogiendo el valor de media o mediana.        

In [9]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes
# Informción de la columna 'days_employed'

credit_scoring['days_employed'].isna().sum()

2174

In [10]:
# Informción de la columna 'days_employed'

credit_scoring['days_employed'].dtypes

dtype('float64')

In [11]:
# Informción de la columna 'total_income'

credit_scoring['total_income'].isna().sum()

2174

In [12]:
credit_scoring['total_income'].dtypes

dtype('float64')

In [13]:
# Comprobación de la distribución (de datos ausentes en 'total_income' y "days_employed")

len(credit_scoring[credit_scoring["days_employed"].isna()]) == len(credit_scoring[credit_scoring["total_income"].isna()])

True

In [14]:
credit_scoring_dropna = credit_scoring.dropna()

In [15]:
list(credit_scoring.columns)

['children',
 'days_employed',
 'dob_years',
 'education',
 'education_id',
 'family_status',
 'family_status_id',
 'gender',
 'income_type',
 'debt',
 'total_income',
 'purpose']

In [16]:
credit_scoring["education"].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [17]:
credit_scoring_dropna["education"].value_counts()

secondary education    12342
bachelor's degree       4222
SECONDARY EDUCATION      705
Secondary Education      646
some college             613
BACHELOR'S DEGREE        251
Bachelor's Degree        243
primary education        231
Some College              40
SOME COLLEGE              22
PRIMARY EDUCATION         16
Primary Education         14
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [18]:
credit_scoring["family_status"].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [19]:
credit_scoring_dropna["family_status"].value_counts()

married              11143
civil partnership     3735
unmarried             2525
divorced              1083
widow / widower        865
Name: family_status, dtype: int64

In [20]:
credit_scoring["gender"].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [21]:
credit_scoring_dropna["gender"].value_counts()

F      12752
M       6598
XNA        1
Name: gender, dtype: int64

In [22]:
credit_scoring["income_type"].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [23]:
credit_scoring_dropna["income_type"].value_counts()

employee                       10014
business                        4577
retiree                         3443
civil servant                   1312
unemployed                         2
student                            1
entrepreneur                       1
paternity / maternity leave        1
Name: income_type, dtype: int64

Al parecer, si no existe dato de experiencia laboral, tampoco lo hay de ingresos. Tiene sentido, pues los ingresos están directamente relacionados con la experiencia laboral.

# Etapa 2. Transformación de datos <a id='data_preprocessing'></a>


## Análisis de datos por columnas

### Análisis de datos en la columna "education" (la educación del cliente)

In [26]:
# Identificación de valores únicos en la columna "education"
credit_scoring["education"].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [27]:
# Correción de valores en la columna "education" (educación de los clientes)
credit_scoring["education"] = credit_scoring["education"].str.lower()

In [28]:
# Comprobación de valores corregidos en la columna "education" (educación de los clientes)
credit_scoring["education"].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

### Análisis de datos en la columna "children" ( el número de hijos en la familia)

In [29]:
# Distribución de los valores en la columna `children`
credit_scoring["children"].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [30]:
#  Cálculo del porcentaje de datos problemáticos en la columna `children`

veinte_hijos = len(credit_scoring[credit_scoring["children"] == 20])
hijos_negativos = len(credit_scoring[credit_scoring["children"] == -1]) 
total_datos = len(credit_scoring)
datos_problematicos = ((veinte_hijos+hijos_negativos)/total_datos)*100
datos_problematicos

0.5714285714285714

* Existen 47 registros problemáticos que indican un valor negativo de hijos (-1) y 76 registros con reporte de 20 hijos (0.57 % de datos con problemas). Puede ser un error de digitación. Se reemplazarán los valores "-1" por "1" y "20" por "2".

In [31]:
# Corrección de datos en la columna "children" 

#credit_scoring.drop(credit_scoring[credit_scoring['children'] == 20].index, inplace = True)
#credit_scoring.drop(credit_scoring[credit_scoring['children'] == -1].index, inplace = True)
#credit_scoring.reset_index()
#credit_scoring.info()

credit_scoring['children'] = credit_scoring['children'].replace(20, 2)
credit_scoring['children'] = credit_scoring['children'].replace(-1, 1)

In [32]:
# Comprobación de datos corregidos en la columna "chilren" (nùmero de hijos)

credit_scoring["children"].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Análisis de datos en la columna "days_employed" (experiencia laboral en días)

Para la columna "days_employed" ya se identificaron los problemas: los datos de experiencia laboral están días (y no en años que es lo usual), tienen valores negativos, y de coma flotante.


In [33]:
# Porcentaje de datos negativos en la columna "days_employed"

negative_records = len(credit_scoring[credit_scoring["days_employed"]<0])
total_records = len(credit_scoring["days_employed"])
percentage_negative_records = (negative_records/total_records)*100
percentage_negative_records

73.89547038327527

In [34]:
# Porcentaje datos ausentes en days_employed

ausentes_experiencia = len(credit_scoring[credit_scoring["days_employed"].isna()])
total_valores = len(credit_scoring["days_employed"])
porcentaje_ausentes_experiencia = (ausentes_experiencia/total_valores)*100 
porcentaje_ausentes_experiencia                                       

10.099883855981417

In [35]:
# Porcentaje datos mayores a 40 años de experiencia en days_employed

extreme_records = len(credit_scoring[credit_scoring["days_employed"]>14600])
total_records = len(credit_scoring["days_employed"])
percentage_extreme_records = (extreme_records/total_records)*100
percentage_extreme_records

16.004645760743323

La cantidad de datos negativos es del 73%. Es un valor muy alto, por lo que no se podrán eliminar las filas problemáticas. Este es un error técnico, y lo más probable es que los datos sean positivos y enteros. Existe un 16 % de datos muy altos, personas con más de 40 años de experiencia, lo cual es improbable. Estos errores se corregirán en esta sección. También existe un 10 % de valores ausentes (esto se tratará en la siguiente sección).

In [36]:
#  Transformación a valores enteros

credit_scoring["days_employed"]  = credit_scoring["days_employed"].fillna(0)
credit_scoring["days_employed"] = credit_scoring["days_employed"].astype(float)
credit_scoring["days_employed"] = credit_scoring["days_employed"].astype(int)
credit_scoring["days_employed"] = credit_scoring["days_employed"].replace(0, float('nan'))

In [37]:
# Comprobación de corrección de valores a enteros

credit_scoring["days_employed"].head(20)

0      -8437.0
1      -4024.0
2      -5623.0
3      -4124.0
4     340266.0
5       -926.0
6      -2879.0
7       -152.0
8      -6929.0
9      -2188.0
10     -4171.0
11      -792.0
12         NaN
13     -1846.0
14     -1844.0
15      -972.0
16     -1719.0
17     -2369.0
18    400281.0
19    -10038.0
Name: days_employed, dtype: float64

In [38]:
# Corrección valores extremos 
credit_scoring.loc[credit_scoring.days_employed > 0, "days_employed"] = float('nan')

In [39]:
# Comprobación corrección valores extremos 
credit_scoring["days_employed"].head(20)

0     -8437.0
1     -4024.0
2     -5623.0
3     -4124.0
4         NaN
5      -926.0
6     -2879.0
7      -152.0
8     -6929.0
9     -2188.0
10    -4171.0
11     -792.0
12        NaN
13    -1846.0
14    -1844.0
15     -972.0
16    -1719.0
17    -2369.0
18        NaN
19   -10038.0
Name: days_employed, dtype: float64

In [40]:
# Corrección valores negativos
credit_scoring["days_employed"] = credit_scoring["days_employed"].abs()

In [41]:
# Comprobación corrección valores negativos
credit_scoring["days_employed"].head(20)

0      8437.0
1      4024.0
2      5623.0
3      4124.0
4         NaN
5       926.0
6      2879.0
7       152.0
8      6929.0
9      2188.0
10     4171.0
11      792.0
12        NaN
13     1846.0
14     1844.0
15      972.0
16     1719.0
17     2369.0
18        NaN
19    10038.0
Name: days_employed, dtype: float64

### Análisis de datos en la columna "dob_years " (la edad del cliente en años)

In [42]:
# Análisis de valores extraños y ausentes
cero_clients = len(credit_scoring[credit_scoring["dob_years"]==0])
cero_clients

101

In [43]:
#  Cálculo del porcentaje de los valores ausentes en comparación con el conjunto de datos completo

cero_clients = len(credit_scoring[credit_scoring["dob_years"]==0])
valores_totales_edad = len(credit_scoring["dob_years"])
porcentaje_ausentes_edad = (cero_clients/valores_totales_edad)*100
porcentaje_ausentes_edad

0.4692218350754936

En este caso, los valores problemáticos son aquellos que representan clientes con edades de "0" (0.46 % de los datos). Se investigará si estos valores pueden ser rellenados con la media o mediana.

In [44]:
#  Matriz filtrada con edades mayores a cero 

edad_mayor_a_cero = credit_scoring[credit_scoring["dob_years"]>0]

In [45]:
#  Cáculo media para columna de edad del cliente

int(edad_mayor_a_cero["dob_years"].mean())

43

In [46]:
#  Cáculo mediana para columna de edad del cliente

edad_mayor_a_cero["dob_years"].median()

43.0

No existe diferencia entre los valores dr media y mediana. El valor con el que se debe trabajar es 43 años. 

In [47]:
# Relleno de valores 

credit_scoring["dob_years"] = credit_scoring["dob_years"].replace(0,43)

In [48]:
#  Verificación de la correción

credit_scoring["dob_years"].sort_values()

766      19
8316     19
4098     19
9218     19
10235    19
         ..
19642    74
2557     74
11532    74
3460     74
8880     75
Name: dob_years, Length: 21525, dtype: int64

### Análisis de datos en la columna "family_status" (estado civil)

Al revisar la columna "family status", al parecer, no hay problemas.

In [49]:
# Análisis de la columna "family_status"

credit_scoring["family_status"].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

### Análisis de datos en la columna "gender" (género del cliente)

En la columna "gender" al parecer tampoco hay problemas.

In [50]:
# Análisis de la columna "gender"

credit_scoring["gender"].unique()

array(['F', 'M', 'XNA'], dtype=object)

### Análisis de datos en la columna "income_type" (tipo de empleo)

En la columna "income_type" no hay inconvenientes.

In [51]:
# Análisis de la columna "income_type

credit_scoring["income_type"].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

### Análisis de duplicados en el conjunto datos

Existen 71 duplicados aparentes (exactos) en el conjunto de datos. Estos se eliminarán pues no aportan información adicional.   

In [52]:
# Comprobar los duplicados

credit_scoring.duplicated().sum()

71

In [53]:
# Porcentaje de valores ducplicados

#duplicados = credit_scoring.duplicated().sum()
#porcentaje_duplicados = (duplicados/valores_totales)*100
#porcentaje_duplicados

In [54]:
# Eliminación de duplcadios
credit_scoring = credit_scoring.drop_duplicates().reset_index(drop=True)

In [55]:
# Última comprobación para ver si tenemos duplicados
credit_scoring.duplicated().sum()

0

In [56]:
# Tamñana del conjunto de datos después de la eliminación de duplciados

credit_scoring.shape

(21454, 12)

### Análisis de la columna "purpose" (propósito del crédito)

In [57]:
credit_scoring["purpose"].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [58]:
# Función que reemplaza nombres incorrectos por los corregidos

wrong_values = credit_scoring["purpose"]

def replace_wrong_values(wrong_values):
    
    for purpose in wrong_values:       
         if 'house' in purpose:
            credit_scoring["purpose"] = credit_scoring["purpose"].replace(purpose, "real state")
         elif 'real estate' in purpose:
            credit_scoring["purpose"] = credit_scoring["purpose"].replace(purpose, "real state")
         elif 'housing' in purpose:
            credit_scoring["purpose"] = credit_scoring["purpose"].replace(purpose, "real state")
         elif 'property' in purpose:
            credit_scoring["purpose"] = credit_scoring["purpose"].replace(purpose, "real state")
         elif 'car' in purpose:
            credit_scoring["purpose"] = credit_scoring["purpose"].replace(purpose, "car") 
         elif 'education' in purpose:
            credit_scoring["purpose"] = credit_scoring["purpose"].replace(purpose, "education")
         elif 'educated' in purpose:
            credit_scoring["purpose"] = credit_scoring["purpose"].replace(purpose, "education")
         elif 'university' in purpose:
            credit_scoring["purpose"] = credit_scoring["purpose"].replace(purpose, "education")   
         elif 'wedding' in purpose:
            credit_scoring["purpose"] = credit_scoring["purpose"].replace(purpose, "wedding")

replace_wrong_values(wrong_values)

In [59]:
# Comprueba valores únicos (corrección de nombres) en columna "purpose"

credit_scoring["purpose"].unique()

array(['real state', 'car', 'education', 'wedding'], dtype=object)

In [60]:
# Función alternativa que reemplaza nombres incorrectos por los corregidos

#def replace_wrong_values(wrong_values, correct_value): # pasamos un diccionario de valores incorrectos y una string con el valor correcto en la entrada de la función
#    for wrong_value in wrong_values: # un bucle sobre nombres mal escritos
#        credit_scoring["purpose"] = credit_scoring["purpose"].replace(wrong_value, correct_value) # llamamos a replace() por cada nombre incorrecto

# Nombres incorrectos

#duplicates_1 = ['purchase of the house','housing transactions','purchase of the house for my family',
#               'buy real estate','construction of own property','property','building a property',
#               'transactions with commercial real estate','building a real estate','housing','transactions with my real estate',
#               'purchase of my own house','real estate transactions','buying property for renting out',
#               'housing renovation','buy commercial real estate','buy residential real estate',] 
#duplicates_2 = ['car purchase','buying a second-hand car','buying my own car','cars','second-hand car purchase',
#               'to own a car','purchase of a car','to buy a car','real estate',]
#duplicates_3 = ['supplementary education','to become educated','getting an education','to get a supplementary education',
#               'getting higher education','profile education','going to university','university education']
#duplicates_4 = ['to have a wedding','wedding ceremony','having a wedding']

# Nombres correctos

#name_1 = 'real state' 
#name_2 = 'car'
#name_3 = 'education'
#name_4 = 'wedding' 

# Llamamos a la función

#replace_wrong_values(duplicates_1, name_1) 
#replace_wrong_values(duplicates_2, name_2)
#replace_wrong_values(duplicates_3, name_3)
#replace_wrong_values(duplicates_4, name_4)

El nuevo conjunto de datos tiene los siguientes cambios, en orden de relevancia:

1. Se eliminaron los registros que contenían errores en la cantidad de hijos "children" (negativos y 20 hijos). Esto corresponde a un 0.57 % de datos eliminados.

2. En el campo de edad del cliente ("dob_years") se eliminaron los registros con datos de valor "0" (0.46 % de los datos).

3. Se eliminaron los registros duplicados (0.33 % de los datos).

4. Los registros de educación se transformaron a letras minúsculas.

5. Se corrigieron los duplicados implícitos de la columna de próposito del crédito "purpose", para tener únicamente cuatro categorías representativas: 'real state', 'car', 'education', 'wedding'. 

## Tratamiento de  valores ausentes

In [61]:
# Encuentra los diccionarios

education = {                    
    0: "bachelor's degree", 
    1: "secondary education",    
    2: "some college", 
    3: "primary education",
    4: "graduate degree"
} 

family = {                    
    0: "married", 
    1: "civil partnership",    
    2: "widow / widower", 
    3: "divorced",
    4: "unmarried"
}


### Restaurar valores ausentes en `total_income`

Las columnas ingreso mensual ("total_income") y experiencia laboral en días ("days_employed") tienen valores ausentes. Para rellenar los valores, se trabajará categorizando variables que expliquen los datos de estas columnas, y se aplicará la media o mediana por categorías para el relleno.  

En ese sentido, se empezará por abordar los valores ausentes del ingreso total. Se creará una categoría de edad para los clientes en una nueva columna. Esta estrategia facilitará el cálculo de valores para el ingreso total. 

In [62]:
# Función que calcula la categoría de edad

def age_group(age):
    """
    The function returns the age group according to the age value, using the following rules:
    —'young_adult' for age <= 30
    —'adult' for 19 <= age <= 60
    —'retired' for all other cases
    """
    
    if age <= 30:
        return 'young_adult'
    if age <= 60:
        return 'adult'
    return 'retired' 
    

In [63]:
# Prueba si la función funciona bien
#age_group(20) 
#age_group(55)
age_group(70) 

'retired'

In [64]:
# Nueva columna basada en la función

credit_scoring['age_group'] = credit_scoring['dob_years'].apply(age_group)


In [65]:
# Comprueba los valores en la nueva columna

credit_scoring.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,real state,adult
1,1,4024.0,36,secondary education,1,married,0,F,employee,0,17932.802,car,adult
2,0,5623.0,33,secondary education,1,married,0,M,employee,0,23341.752,real state,adult
3,3,4124.0,32,secondary education,1,married,0,M,employee,0,42820.568,education,adult
4,0,NaN,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,adult
5,0,926.0,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,real state,young_adult
6,0,2879.0,43,bachelor's degree,0,married,0,F,business,0,38484.156,real state,adult
7,0,152.0,50,secondary education,1,married,0,M,employee,0,21731.829,education,adult
8,2,6929.0,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding,adult
9,0,2188.0,41,secondary education,1,married,0,M,employee,0,23108.150,real state,adult


Los factores que influyen en los ingresos son: 1) edad, 2) educación, 3) fuente de ingresos, 4) estado civil, y 5) género. Se investigará como esta variables afectan el ingreso, y para simplificar los cálculos, se escogerá la variable que tenga más incidencia para el relleno.   

Para empezar, se creará una tabla que solo tenga datos sin valores ausentes. Estos datos se utilizarán para restaurar los valores ausentes.  

In [66]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas 

credit_scoring_dropna = credit_scoring.dropna()
credit_scoring_dropna.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,real state,adult
1,1,4024.0,36,secondary education,1,married,0,F,employee,0,17932.802,car,adult
2,0,5623.0,33,secondary education,1,married,0,M,employee,0,23341.752,real state,adult
3,3,4124.0,32,secondary education,1,married,0,M,employee,0,42820.568,education,adult
5,0,926.0,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,real state,young_adult
6,0,2879.0,43,bachelor's degree,0,married,0,F,business,0,38484.156,real state,adult
7,0,152.0,50,secondary education,1,married,0,M,employee,0,21731.829,education,adult
8,2,6929.0,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding,adult
9,0,2188.0,41,secondary education,1,married,0,M,employee,0,23108.150,real state,adult
10,2,4171.0,36,bachelor's degree,0,married,0,M,business,0,18230.959,real state,adult


### Análisis de ingresos en función de grupo de edad

Se espera que a mayor edad, los ingresos sean mayores.

In [67]:
# Examina los valores medios de los ingresos en función de los factores que identificaste (en este caso grupo de edad)

credit_scoring_dropna.groupby('age_group')['total_income'].mean()

age_group
adult          28335.037107
retired        29510.907682
young_adult    25840.450402
Name: total_income, dtype: float64

In [68]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste (en este caso grupo de edad)

credit_scoring_dropna.groupby('age_group')['total_income'].median()

age_group
adult          24588.957
retired        25501.547
young_adult    22976.210
Name: total_income, dtype: float64

Efectivamente, los adultos ganan mas que los jóvenes adultos. Sin embargo, los retirados ganan menos que estos dos grupos, lo cual tiene sentido también puesto que aunque tengan más experiencia este grupo salió del mercado laboral. 

### Análisis de ingresos en función de educación

En este caso se espera que a mayor nivel educativo, los ingresos sean mayores.

In [69]:
# Análisis de valores medios de ingresos en función de educación

credit_scoring_dropna.groupby('education')['total_income'].mean()


education
bachelor's degree      33866.375230
graduate degree        27772.929500
primary education      22990.762982
secondary education    25498.606991
some college           29423.635397
Name: total_income, dtype: float64

In [70]:
# Análisis de valores medianos de ingresos en función de educación
credit_scoring_dropna.groupby('education')['total_income'].median()

education
bachelor's degree      28744.4340
graduate degree        25161.5835
primary education      20823.9590
secondary education    22673.4220
some college           26119.0075
Name: total_income, dtype: float64

Con esta agrupación se detectan algunas inconsistencias en los datos. Por ejemplo, las personas con títulos de posgrado ("graduate degree") ganan menos que aquellos que tienen título de pregrado (education
bachelor's degree). Otro ejemplo, las personas con algo de universidad ("some college") ganan más que los que tienen títulos. Al parecer, los datos de educación de los clientes no son confiables, por lo que se recomienda no trabajar con los mismos.

### Análisis de ingresos en función de fuente de ingresos

En este caso se espera que emprendedores y trabajadores del sector privado ganen más que el resto de grupos.

In [71]:
# Análisis de valores medios de ingresos en función de tipo de ingreso
credit_scoring_dropna.groupby('income_type')['total_income'].mean()

income_type
business                       32386.793835
civil servant                  27343.729582
employee                       25820.841683
entrepreneur                   79866.103000
paternity / maternity leave     8612.661000
student                        15712.260000
Name: total_income, dtype: float64

In [72]:
# Análisis de valores medianos de ingresos en función de tipo de ingreso
credit_scoring_dropna.groupby('income_type')['total_income'].median()

income_type
business                       27577.2720
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
student                        15712.2600
Name: total_income, dtype: float64

Del análisis de este grupo, se desprende que los datos son confiables y que se puede trabajar con los mismos pues describen adecaudamente los ingresos en función de la fuente de los mismos.

### Análisis de valores medios de ingresos en función de estado civil

De este análisis es difícil anticipar resultados.  

In [73]:
# Análisis de valores medianos de ingresos en función de estado civil
credit_scoring_dropna.groupby('family_status')['total_income'].median()

family_status
civil partnership    23985.7415
divorced             24653.3380
married              24372.2510
unmarried            23843.2035
widow / widower      22570.8710
Name: total_income, dtype: float64

Aparente no existen diferencias significativas en los ingresos en función del estado civil. 

### Análisis de valores medios de ingresos en función del género

De este análisis se espera que los hombres ganen más que las mujeres (lamentablemente...). 

In [74]:
# Análisis de valores medios de ingresos en función de género
credit_scoring_dropna.groupby('gender')['total_income'].mean()

gender
F      25560.371100
M      31636.394768
XNA    32624.825000
Name: total_income, dtype: float64

In [75]:
# Análisis de valores medios de ingresos en función de género
credit_scoring_dropna.groupby('gender')['total_income'].median()

gender
F      22210.6180
M      27526.4765
XNA    32624.8250
Name: total_income, dtype: float64

Se confirma lo esperado. Se desconoce el significado de "XNA". 

Del análisis se desprende que la característica que mejor define los ingresos es la fuente de los mismos. Para esta característica, existen varias categorías en las que se aprecian diferencias aparentemente significativas entre las mismas y consistentes con lo que esperaba en cuanto a ingresos.  

Al parecer, existen valores extremos, por lo tanto se usará el valor de la mediana.


In [76]:
#credit_scoring['total_income'].head(20)
credit_scoring['total_income'].isna().sum()

2103

In [78]:
#  Completar los valores ausentes en la colunma "total_income"

total_income_median = credit_scoring.groupby('income_type')['total_income'].transform('median')
total_income_median.head(20)
credit_scoring['total_income'] = credit_scoring['total_income'].fillna(total_income_median)

In [79]:
# Comprobación relleno de datos en "total_income"

credit_scoring['total_income'].isna().sum()
#credit_scoring['total_income'].head(20)

0

###  Restaurar valores en `days_employed`

Para restaurar los valores en "days_employed" se empleará una categorización con los datos de "education".

In [230]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados

credit_scoring_dropna.groupby('income_type')['days_employed'].median()

income_type
business                       1547.0
civil servant                  2689.0
employee                       1573.5
entrepreneur                    520.0
paternity / maternity leave    3296.0
student                         578.0
Name: days_employed, dtype: float64

In [231]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados

credit_scoring_dropna.groupby('income_type')['days_employed'].mean()

income_type
business                       2111.031899
civil servant                  3399.400915
employee                       2325.996904
entrepreneur                    520.000000
paternity / maternity leave    3296.000000
student                         578.000000
Name: days_employed, dtype: float64

In [232]:
credit_scoring['days_employed'].head(20)
#credit_scoring['days_employed'].isna().sum()

0      8437.0
1      4024.0
2      5623.0
3      4124.0
4         NaN
5       926.0
6      2879.0
7       152.0
8      6929.0
9      2188.0
10     4171.0
11      792.0
12        NaN
13     1846.0
14     1844.0
15      972.0
16     1719.0
17     2369.0
18        NaN
19    10038.0
Name: days_employed, dtype: float64

In [233]:
#  Completar los valores ausentes en la columna "days_employed

days_employed_median = credit_scoring.groupby('education')['days_employed'].transform('median')
credit_scoring['days_employed'] = credit_scoring['days_employed'].fillna(days_employed_median)

Se utilizarán las medianas, porque la parecer existen valores extremos.

In [234]:
# Comprobación relleno de datos en "total_income"

credit_scoring['days_employed'].isna().sum()
# credit_scoring['days_employed'].head(20)

0

## Etapa 3. Clasificación de datos <a id='data_classification'></a>

Para responder las  preguntas de investigación, la única variable que requiere categorizacion es la de ingresos. 


In [134]:
# Revisar todos los datos numéricos en la columna seleccionada para la clasificación

credit_scoring['total_income'].isna().sum()

0

In [135]:
# Obtener estadísticas resumidas para la columna
credit_scoring['total_income'].describe()


count     21525.000000
mean      26436.051922
std       15686.986477
min        3306.762000
25%       17247.708000
50%       22815.103500
75%       31287.991000
max      362496.645000
Name: total_income, dtype: float64

Se utlizarán tres rangos para agrupar:

1. 3306 <= income < 17219 ("low_income")
2. 17219 <= income < 31331 ("medium_income")
3. income >=  31331 ("high_income")



In [237]:
# Función para clasificar en diferentes grupos numéricos basándose en rangos

def income_debt(row):
    
    """"
    Los rangos se basan en las estadísticas de la columna 
    y la realidad socioeconómica de USA (de donde probablemente vienen los datos).
    
    """
 
    income = row['total_income']
    debt = row['debt']

    if  3306 <= income < 17219:
        return 'low_income'
    if  17219  <= income < 31331:
        return 'medium_income'
    if  income >= 31331:
        return 'high_income'



In [238]:
# Crear una columna con categorías

credit_scoring['income_categories'] = credit_scoring.apply(income_debt, axis=1) 

In [239]:
# Contar los valores de cada categoría para ver la distribución
credit_scoring['income_categories'].value_counts()

medium_income    10726
high_income       5365
low_income        5363
Name: income_categories, dtype: int64

## Etapa 4. Comprobación de las hipótesis <a id='hypotheses'></a>


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [139]:
# Porcentaje de incumplimiento en función del número de hijos

credit_scoring.pivot_table(index="children", values='debt', aggfunc='mean') 

,debt
children,
0,0.075129
1,0.091470
2,0.094791
3,0.081818
4,0.097561
5,0.000000


In [241]:
#Número de clientes que incumplen en función de la cantidad de hijos

credit_scoring.pivot_table(index='children', values='debt', aggfunc='sum', margins=True) 

,debt
children,
0,1063
1,445
2,202
3,27
4,4
5,0
All,1741


**Conclusión**

El 7.5 % de los clientes sin hijos tienen deudas. En relación al conjunto de clientes que mantienen deudas, esto corresponde a 1063 de 1741 clientes (la mayoría). Es decir, las personas con hijos son mejores clientes.  


**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [242]:
# Porcentaje de incumplimiento en función del estado civil

credit_scoring.pivot_table(index="family_status", values='debt', aggfunc='mean') 

,debt
family_status,
civil partnership,0.093471
divorced,0.071130
married,0.075452
unmarried,0.097509
widow / widower,0.065693


In [244]:
#Número de clientes que incumplen en función del estado civil

credit_scoring.pivot_table(index="family_status", values='debt', aggfunc='sum',margins=True) 

,debt
family_status,
civil partnership,388
divorced,85
married,931
unmarried,274
widow / widower,63
All,1741


**Conclusión**

La tasa de incumplimiento de personas casadas es del 7.5%. Esto corresponde a 931 de 1741 clientes que mantienen deuda (la mayoría). Esto implica que las personas casadas son el grupo de más alto riesgo. 

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [149]:
# Porcentaje de incumplimiento en función del ingreso

credit_scoring.pivot_table(index="income_categories", values='debt', aggfunc='mean') 

,debt
income_categories,
high_income,0.071389
low_income,0.079620
medium_income,0.086228


In [245]:
#Número de clientes que incumplen en función del ingreso

credit_scoring.pivot_table(index="income_categories", values='debt', aggfunc='sum',margins=True) 

,debt
income_categories,
high_income,383
low_income,427
medium_income,931
All,1741


**Conclusión**

La tasa de incumplimiento en la clase media es del 8.6 %. Esto corresponde a 931 de 1741 clientes con deuda (la mayoría). En ese sentido, este grupo socio-económico es el de mayor reisgo para otorgar préstamos. 

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [246]:
# Porcentaje de incumplimiento en función del propósito del crédito

credit_scoring.pivot_table(index="purpose", values='debt', aggfunc='mean') 

,debt
purpose,
car,0.093590
education,0.092200
real state,0.072334
wedding,0.080034


In [247]:
#Número de clientes que incumplen en función del propósito del crédito


credit_scoring.pivot_table(index="purpose", values='debt', aggfunc='sum',margins=True) 

,debt
purpose,
car,403
education,370
real state,782
wedding,186
All,1741


**Conclusión**

En orden de mayor a menor incumplimiento se determina que asociado al propósito del crédito la variación de la tasa de incumplimiento es: bienes raíces (782 clientes que representan 7.2 % de deudores en este segmento), adquisión de vehículos (403 clientes representando el 9.3 % de personas que tienen deuda por este concepto), gastos educativos (370 clientes equivalentes a una tasa de 9.2 % de clientes con deudas por educación), y bodas (186 clientes que representan una tasa de 8.0 % de deudores por bodas). En ese sentido, los créditos de consumo para bodas o afines son los más seguros.


# Etapa 5. Conclusión general <a id='end'></a>

Con respecto al procesamiento de datos las conclusiones son las siguientes:

1. La columna de educación requirió una transformación a letras minúsculas para facilitar la categorización. 
2. La columna de número de hijos contaba con registros negativos (-1) de muchos hijos (20). Estos registros fueron eliminados (0.57% % de los datos). 
3. En los datos de edad del cliente, se encontraron 0.46 % de registros con el valor de 0. Los mismos fueron eliminados. 
4. Los datos de experiencia laboral fueron convertidos a valores positivos, enteros, y rellenados con la mediana en función de categorías de fuentes de ingreso.



Por otra parte, con respecto a las preguntas se investigación se concluye lo siguiente:

1. El 7.5 % de los clientes sin hijos tienen deudas. En relación al conjunto de clientes que mantienen deudas, esto corresponde a 1063 de 1741 clientes (la mayoría). Es decir, las personas con hijos son mejores clientes. 
2. La tasa de incumplimiento de personas casadas es del 7.5%. Esto corresponde a 931 de 1741 clientes que mantienen deuda (la mayoría). Esto implica que las personas casadas son el grupo de más alto riesgo.
3. La tasa de incumplimiento en la clase media es del 8.6 %. Esto corresponde a 931 de 1741 clientes con deuda (la mayoría). En ese sentido, este grupo socio-económico es el de mayor reisgo para otorgar préstamos.
4. En orden de mayor a menor incumplimiento se determina que asociado al propósito del crédito la variación de la tasa de incumplimiento es: bienes raíces (782 clientes que representan 7.2 % de deudores en este segmento), adquisión de vehículos (403 clientes representando el 9.3 % de personas que tienen deuda por este concepto), gastos educativos (370 clientes equivalentes a una tasa de 9.2 % de clientes con deudas por educación), y bodas (186 clientes que representan una tasa de 8.0 % de deudores por bodas).En ese sentido, los créditos de consumo para bodas o afines son los más seguros.
